# Test Case

## Test case for combo issue

In [ ]:
combo_issue_supervisor_id_obj = list(set(ComboIssueRequest.objects.all().values_list('supervisor_id', flat=True)))
print(combo_issue_supervisor_id_obj)

for supervisor_id in combo_issue_supervisor_id_obj:
#     print(supervisor_id)
    if UserHierarchyMap.objects.filter(subordinate=supervisor_id).exists():
        superior_id = UserHierarchyMap.objects.get(subordinate=supervisor_id).superior_id
        print(superior_id)
    else:
        if supervisor_id in (22,26):
            superior_id = 25
            print('changed')

    combo_issue_request_supervisor_obj = ComboIssueRequest.objects.filter(supervisor_id=supervisor_id).exclude(senior_supervisor_id__isnull=True)
    for combo_issue in combo_issue_request_supervisor_obj:
        print(combo_issue.senior_supervisor_id)
        if combo_issue.senior_supervisor_id == 79:
            print('to change')
            senior_superior_id = UserHierarchyMap.objects.get(subordinate=supervisor_id).superior_id
            print('to change', senior_superior_id)
            combo_issue.senior_supervisor_id = senior_superior_id
            combo_issue.save()

## check farmer code test case

In [ ]:
farmer_obj = FarmerClusterSeasonMap.objects.filter(season_id=3)


for code in FarmerClusterSeasonMap.objects.filter(season_id=3):
    farmer_code = code.seasonal_farmer_code
    
    farmer_cluster_letter = FarmerClusterSeasonMap.objects.get(season_id=3, seasonal_farmer_code=farmer_code).cluster.name[:1]

    farmer_supervisor_letter = UserFarmerMap.objects.get(farmer__season_id=3, farmer__seasonal_farmer_code=farmer_code).officer.username[:1]
    agent_id = AgentFarmerMap.objects.get(farmer__seasonal_farmer_code=farmer_code).agent_id
    
    farmer_unicode_letter = UserClusterMap.objects.get(season_id=3, user_id=agent_id).unique_code

    gen_farmer_code_letters = farmer_cluster_letter.upper() + farmer_supervisor_letter.upper() + farmer_unicode_letter.upper()

    split_from_farme_code = farmer_code[2:5]

    if not gen_farmer_code_letters==split_from_farme_code:
        print('not matched',farmer_code, gen_farmer_code_letters, split_from_farme_code)
        


## To check Agent code

In [ ]:
import pandas as pd

code = list(UserProfile.objects.all().values_list('code',flat=True))

dataframe1 = pd.read_excel('/home/steve/Downloads/seed_upload.xlsx') 
for index, row in dataframe1.iterrows():
    agent_code = row['Agent Code']

    if UserProfile.objects.filter(code = agent_code).exists():
        print(agent_code)
        agent_id = UserProfile.objects.get(code = agent_code).user_id
        agent_name = UserProfile.objects.get(code = agent_code).user.first_name
        print(agent_id)
        print(agent_name)
        print(row['Date'])
        print(row['No of Packets 100'])
        print('***********************************')

## To check matching agents between cluster and supervisor 

In [ ]:
supervisor_ids = list(UserClusterMap.objects.filter(season_id=3, user__userprofile__user_type_id=5).values_list('user_id', flat=True))

for supervisor_id in supervisor_ids:
    print(supervisor_id)
    
    agent_ids_under_supervisor = list(AgentSupervisorSeasonMap.objects.filter(season_id=3, supervisor_id=supervisor_id).values_list('agent_id', flat=True))
#     print(agent_ids_under_supervisor)

    cluster_id = UserClusterMap.objects.get(season_id=3, user_id=supervisor_id).cluster_id
#       print(cluster_id)
    
    agent_ids_under_cluster = list(UserClusterMap.objects.filter(season_id=3, cluster_id=cluster_id).exclude(user_id=supervisor_id).values_list('user_id', flat=True))
#     print(agent_ids_under_cluster)
    
    if agent_ids_under_supervisor.sort() == agent_ids_under_cluster.sort():
        print('matched')

## To check farmer count under supervisor and cluster

In [ ]:
season_id =3

cluster_ids = list(ClusterSeasonMap.objects.filter(season_id=season_id).values_list('cluster_id', flat=True))

for cluster_id in cluster_ids:
    farmer_cluster_obj = FarmerClusterSeasonMap.objects.filter(season_id=season_id, cluster_id=cluster_id).count()
    farmer_cluster_count = farmer_cluster_obj
    
    supervisor_ids = UserClusterMap.objects.filter(season_id=3,cluster_id=cluster_id ,user__userprofile__user_type_id=5)
    
    for supervisor_id in supervisor_ids:
        superviosr_farmer_count = UserFarmerMap.objects.filter(farmer__season_id=3, officer_id=supervisor_id.user_id).count()
        supervisor_id = supervisor_id.user_id,


        if farmer_cluster_count != superviosr_farmer_count:
            print('cluster_id:',cluster_id)
            print('farmer_cluster_count:',farmer_cluster_count)
            print('supervisor_id:', supervisor_id)
            print('superviosr_farmer_count:',superviosr_farmer_count)
            print('***********************************************************')
            
            


## update farmer unmatched_code 

In [ ]:
import pandas as pd


farmer_obj = FarmerClusterSeasonMap.objects.filter(season_id=3)

codes=[]
master_dict={}
for code in FarmerClusterSeasonMap.objects.filter(season_id=3):
    farmer_code = code.seasonal_farmer_code
    
    farmer_cluster_letter = FarmerClusterSeasonMap.objects.get(season_id=3, seasonal_farmer_code=farmer_code).cluster.name[:1]

    farmer_supervisor_letter = UserFarmerMap.objects.get(farmer__season_id=3, farmer__seasonal_farmer_code=farmer_code).officer.username[:1]
    agent_id = AgentFarmerMap.objects.get(farmer__seasonal_farmer_code=farmer_code).agent_id
    
    farmer_unicode_letter = UserClusterMap.objects.get(season_id=3, user_id=agent_id).unique_code

    gen_farmer_code_letters = farmer_cluster_letter.upper() + farmer_supervisor_letter.upper() + farmer_unicode_letter.upper()

    split_from_farme_code = farmer_code[2:5]

    if not gen_farmer_code_letters==split_from_farme_code:
#         print('not matched',farmer_code, gen_farmer_code_letters, split_from_farme_code)
        codes.append(farmer_code)

In [ ]:
import pandas as pd
df =pd.DataFrame(codes,columns=['farmer_unmatched_codes'])

df.to_excel(str("static/media/") + "farmer_codes.xlsx", engine="xlsxwriter")


In [ ]:
def generate_farmer_code(cluster_id, agent_id, officer_id, season_id):
    season = Season.objects.get(id=season_id).year
    season_name = str(season.strftime("%y"))
    cluster_name = Cluster.objects.get(id=cluster_id).name.upper()
    agent_name = UserClusterMap.objects.get(user_id=agent_id, season_id=season_id).unique_code.upper()
    officer_name = User.objects.get(id=officer_id).username[0:1].upper()
    prefix = season_name + cluster_name[0] + officer_name[0] + agent_name[0]
    print(prefix)
    if IdBank.objects.filter(purpose="farmer", prefix_code=prefix).exists():
        last_code = IdBank.objects.get(purpose="farmer", prefix_code=prefix).last_count
        IdBank.objects.filter(purpose="farmer", prefix_code=prefix).update(
            last_count=last_code + 1
        )
    else:
        last_code = 0
        id_bank_obj = IdBank(purpose="farmer", prefix_code=prefix, last_count=1, business_id=1)
        id_bank_obj.save()
    generated_code_number = last_code + 1
    suffix = str(generated_code_number).zfill(3)
    code = str(prefix) + suffix
    return code

In [ ]:
for index,row in df.iterrows():
    code = row['farmer_unmatched_codes']

    cluster_obj_id = FarmerClusterSeasonMap.objects.get(season_id=3, seasonal_farmer_code=code).cluster.id
    #     print(cluster_obj_id)
    agent_id_obj = AgentFarmerMap.objects.get(farmer__seasonal_farmer_code=code).agent_id
    #     print(agent_id_obj)
    officer_obj_id = UserClusterMap.objects.get(season=3, cluster_id=cluster_obj_id, user__userprofile__user_type_id=5).user.id
    #     print(officer_obj_id)
    season_id=3

    new_code = generate_farmer_code(cluster_obj_id, agent_id_obj, officer_obj_id, season_id,)
    #     print(new_code)
    if FarmerClusterSeasonMap.objects.filter(seasonal_farmer_code=code).exists():
    #     print('yes')
        farmer_cluster_obj = FarmerClusterSeasonMap.objects.filter(seasonal_farmer_code=code,).update(seasonal_farmer_code=new_code)
    #     print('done')
    print('old_code:',code )
    print('new:',new_code)
    print('*********************')

## cluster and supervisor

In [ ]:
super_obj = UserClusterMap.objects.filter(user__username='siddaraju', season_id=3)
for name in super_obj:
    print(name.cluster.name)
    print(name.user.id)

## Agent and Farmer cluster mismatched and update

In [ ]:
agent_ids_obj = list(UserClusterMap.objects.filter(season_id=3, user__userprofile__user_type_id=6).values_list('user_id', flat=True))
master_dict = {}
temp_dict={}
for agent in agent_ids_obj:
    agent_cluster = UserClusterMap.objects.get(season_id=3, user_id=agent).cluster_id
    farmer_ids_obj = list(AgentFarmerMap.objects.filter(agent_id=agent, farmer__season_id=3).values_list('farmer__farmer_id',flat=True))
    for farmer in farmer_ids_obj:
        farmer_cluster = FarmerClusterSeasonMap.objects.get(farmer_id=farmer, season_id=3).cluster_id
        if agent_cluster != farmer_cluster:
            if agent not in master_dict:
                master_dict[agent] = []
            temp_dict['farmer_cluster'] = farmer_cluster
            master_dict[agent].append(farmer)
print(master_dict.keys())
# master_dict

        



In [ ]:
for agent_id in master_dict.keys():
    cluster = UserClusterMap.objects.get(season_id=3, user_id=agent_id).cluster_id
    print('cluster_id', cluster)

    for farmer_id in master_dict[agent_id]:
        print(farmer_id)
        farmer_cluster = FarmerClusterSeasonMap.objects.filter(season_id=3, farmer_id=farmer_id).update(cluster_id = cluster)


## To check farmer count cluster and agent wise

In [ ]:
season_id =3

cluster_ids = list(ClusterSeasonMap.objects.filter(season_id=season_id).values_list('cluster_id', flat=True))
print(cluster_ids)

for cluster_id in cluster_ids:
    cluster_farmer_count = FarmerClusterSeasonMap.objects.filter(season_id=3, cluster_id__in=cluster_ids).count()
print('cluster_farmer_count:',cluster_farmer_count)
    
agent_ids = list(set(UserClusterMap.objects.filter(season_id=3, user__userprofile__user_type_id=6).values_list('user_id', flat=True).order_by('user_id')))
agent_ids

agent_farmer_count = AgentFarmerMap.objects.filter(farmer__season_id=3, agent_id__in=agent_ids).count()
print('agent_farmer_count:',agent_farmer_count)

if cluster_farmer_count != agent_farmer_count:
    print('not matched')
else:
    print('matched')

## To check agent count cluster and supervisor wise

In [ ]:
supervisor_ids = list(UserClusterMap.objects.filter(season_id=season_id, user__userprofile__user_type_id=5).values_list('user_id',flat=True))
supervisor_ids

for supervisor_id in supervisor_ids:
    superviosr_agent_count = AgentSupervisorSeasonMap.objects.filter(season_id=3, supervisor_id=supervisor_id).count()
    #     print('superviosr_agent_count',superviosr_agent_count)
    
    cluster_id = UserClusterMap.objects.get(season_id=3, user_id=supervisor_id).cluster_id
    #     print('cluster_id',cluster_id)
    
    cluster_agent_count = UserClusterMap.objects.filter(season_id=3, cluster_id=cluster_id, user__userprofile__user_type_id=6).count()
    #     print('cluster_agent_count',cluster_agent_count)
    
    if superviosr_agent_count != cluster_agent_count:
        print('count_not matched,', 
              'superviosr_agent_count:', superviosr_agent_count, 
              'cluster_agent_count:',cluster_agent_count)
    else:
        print('matched')

In [3]:
def agent_count_in_cluster_and_supervisor():
    season_id = 3
    try:
        print('To check agent_count_in_cluster_and_supervisor')
        supervisor_ids = list(UserClusterMap.objects.filter(season_id=season_id, user__userprofile__user_type_id=5).values_list('user_id',flat=True))
        supervisor_ids

        for supervisor_id in supervisor_ids:
            superviosr_agent_count = AgentSupervisorSeasonMap.objects.filter(season_id=3, supervisor_id=supervisor_id).count()
            cluster_id = UserClusterMap.objects.get(season_id=3, user_id=supervisor_id).cluster_id
            cluster_agent_count = UserClusterMap.objects.filter(season_id=3, cluster_id=cluster_id, user__userprofile__user_type_id=6).count()
            if superviosr_agent_count != cluster_agent_count:
                print('count_not matched,', 'superviosr_agent_count:', superviosr_agent_count, 'cluster_agent_count:',cluster_agent_count)
        print('Completed')
    except Exception as error:
        print(error)
agent_count_in_cluster_and_supervisor()

To check agent_count_in_cluster_and_supervisor
count_not matched, superviosr_agent_count: 3 cluster_agent_count: 6
Completed


In [12]:
supervisor_ids = list(UserClusterMap.objects.filter(season_id=3, user__userprofile__user_type_id=5).values_list('user_id',flat=True))
supervisor_ids

for supervisor_id in supervisor_ids:
    
    superviosr_agent_count = AgentSupervisorSeasonMap.objects.filter(season_id=3, supervisor_id=supervisor_id).count()
    cluster_id = UserClusterMap.objects.get(season_id=3, user_id=supervisor_id).cluster_id
#     print('cluster_id',cluster_id)
    cluster_agent_count = UserClusterMap.objects.filter(season_id=3, cluster_id=cluster_id, user__userprofile__user_type_id=6).count()
    if superviosr_agent_count != cluster_agent_count:
        print('count_not matched,', 'superviosr_agent_count:', superviosr_agent_count, 'cluster_agent_count:',cluster_agent_count)
        print('supervisor_id', supervisor_id)
        print('cluster_id',cluster_id)
    print('Completed')
       
    

Completed
Completed
Completed
Completed
Completed
count_not matched, superviosr_agent_count: 3 cluster_agent_count: 6
supervisor_id 25
cluster_id 4
Completed
Completed
Completed


In [8]:
# season_id= Season.objects.get(is_active=True).id
season_id=3
def check_test_cases():
    
    if not cluster_supervisor_count():
        result = 'Error_in_Function_1'
        print(result)
#         send_message(result)
    elif not agent_count_in_cluster_and_supervisor():
        result = 'Error_in_Function_2'
        print(result)
#         send_message(result)
    elif not total_cluster_and_superviosr_farmer_count():
        result = 'Error_in_Function_3'
        print(result)
#         send_message(result)
    elif not check_individual_cluster_and_supervior_count():
        result = 'Error_in_Function_4'
        print(result)
#         send_message(result)
    elif not check_individual_superviosr_and_agent_farmer_count():
        result = 'Error_in_Function_5'
        print(result)
#         send_message(result)
    elif not check_agent_and_farmers_are_same_cluster():
        result = 'Error_in_Function_6'
#         send_message(result)
    elif not check_superviosr_and_farmers_are_same_cluster():
        result = 'Error_in_Function_7'
#         send_message(result)
    elif not check_both_agent_and_superviosr_same_cluster():
        result = 'Error_in_Function_8'
#         send_message(result)
    elif not check_seasonal_farmer_code():
        result = 'Error_in_Function_9'
#         send_message(result)
    elif not aadhaar_id_userprofile_count():
        result = 'Error_in_Function_11'
#         send_message(result)
    elif not check_one_agent_in_multiple_cluster():
        result = 'Error_in_Function_12'
#         send_message(result)
    elif not check_farmer_id_in_user_farmer_map():
        result = 'Error_in_Function_13'
#         send_message(result)
    elif not check_farmer_id_in_agent_farmer_map():
        result = 'Error_in_Function_14'
#         send_message(result)
    print('success')


# 1. To check cluster and supervisor count
def cluster_supervisor_count():
    print('To check cluster and supervisor count')
    cluster_count = ClusterSeasonMap.objects.filter(season_id=season_id).count()
    superviosr_count = UserClusterMap.objects.filter(season_id=season_id, user__userprofile__user_type_id=5).count()
    if cluster_count != superviosr_count:
        result = False
    else:
        result = True
        
    return result

# 2. To check agent count in cluster and superviosr
def agent_count_in_cluster_and_supervisor():
    print('To check agent_count_in_cluster_and_supervisor')
    supervisor_ids = list(UserClusterMap.objects.filter(season_id=season_id, user__userprofile__user_type_id=5).values_list('user_id',flat=True))
    supervisor_ids

    for supervisor_id in supervisor_ids:
        superviosr_agent_count = AgentSupervisorSeasonMap.objects.filter(season_id=season_id, supervisor_id=supervisor_id).count()

        cluster_id = UserClusterMap.objects.get(season_id=season_id, user_id=supervisor_id).cluster_id
        cluster_agent_count = UserClusterMap.objects.filter(season_id=season_id, cluster_id=cluster_id, user__userprofile__user_type_id=6).count()
        
        if superviosr_agent_count != cluster_agent_count:
            return False
    return True


# 3. To check Total cluster and superviosr farmer count
def total_cluster_and_superviosr_farmer_count():
    print('To check Total cluster and superviosr farmer count')


    cluster_ids = list(ClusterSeasonMap.objects.filter(season_id=season_id).values_list('cluster_id', flat=True))
    superviosr_ids = list(UserClusterMap.objects.filter(season_id=season_id, user__userprofile__user_type_id=5).values_list('user_id', flat=True))

    total_cluster_farmer_count = FarmerClusterSeasonMap.objects.filter(season_id=season_id, cluster_id__in=cluster_ids).count()
    total_superviosr_farmer_count = UserFarmerMap.objects.filter(farmer__season_id=season_id, officer_id__in=superviosr_ids).count()

    if total_cluster_farmer_count != total_superviosr_farmer_count:
        result = False
    else:
        result = True
    return result

# 4. To check individual cluster and supervior count
def check_individual_cluster_and_supervior_count():
    print('To check individual cluster and supervior count')


    cluster_ids = list(ClusterSeasonMap.objects.filter(season_id=season_id).values_list('cluster_id', flat=True))
    for cluster_id in cluster_ids:
        individual_cluster_farmer_count = FarmerClusterSeasonMap.objects.filter(season_id=season_id, cluster_id=cluster_id).count()

        superviosr_id = UserClusterMap.objects.get(season_id=season_id, cluster_id=cluster_id, user__userprofile__user_type_id=5).user.id

        individual_superviosr_farmer_count = UserFarmerMap.objects.filter(farmer__season_id=season_id, officer_id=superviosr_id).count()

        mis_matched_farmer_count_differences = individual_cluster_farmer_count - individual_superviosr_farmer_count
        
        if mis_matched_farmer_count_differences != 0:
            return  False
    return True
        
# 5. To check individual superviosr and agent farmer count
def check_individual_superviosr_and_agent_farmer_count():
    print('To check individual superviosr and agent farmer count')

    superviosr_ids = list(UserClusterMap.objects.filter(season_id=season_id, user__userprofile__user_type_id=5).values_list('user_id', flat=True))
    for superviosr_id in superviosr_ids:
        individual_superviosr_farmer_count = UserFarmerMap.objects.filter(farmer__season_id=season_id, officer_id=superviosr_id).count()

        agent_ids = list(AgentSupervisorSeasonMap.objects.filter(season_id=season_id, supervisor_id=superviosr_id).values_list('agent_id', flat=True))
        agent_farmer_count = AgentFarmerMap.objects.filter(farmer__season_id=season_id, agent_id__in=agent_ids).count()

        mis_matched_farmer_count_differences = individual_superviosr_farmer_count - agent_farmer_count
        if mis_matched_farmer_count_differences != 0:
            return False
    return True

#  6. To check agent and farmers are same cluster
def check_agent_and_farmers_are_same_cluster():
    print('To check agent and farmers are same cluster')
    agent_ids_obj = list(UserClusterMap.objects.filter(season_id=season_id, user__userprofile__user_type_id=6).values_list('user_id', flat=True))
    temp_dict={}
    for agent in agent_ids_obj:
        agent_cluster = UserClusterMap.objects.get(season_id=season_id, user_id=agent).cluster_id
        farmer_ids_obj = list(AgentFarmerMap.objects.filter(agent_id=agent, farmer__season_id=season_id).values_list('farmer__farmer_id',flat=True))
        for farmer in farmer_ids_obj:
            farmer_cluster = FarmerClusterSeasonMap.objects.get(farmer_id=farmer, season_id=season_id).cluster_id
            if agent_cluster != farmer_cluster:
                return False
    return True

# 7. To check superviosr and farmers are same cluster
def check_superviosr_and_farmers_are_same_cluster():
    print('To check superviosr and farmers are same cluster')
    
    superviosr_ids_obj = list(UserClusterMap.objects.filter(season_id=season_id, user__userprofile__user_type_id=5).values_list('user_id', flat=True))
    temp_dict={}
    for superviosr in superviosr_ids_obj:
        superviosr_cluster = UserClusterMap.objects.get(season_id=season_id, user_id=superviosr).cluster_id
        farmer_ids_obj = list(UserFarmerMap.objects.filter(officer_id=superviosr, farmer__season_id=season_id).values_list('farmer__farmer_id',flat=True))
        for farmer in farmer_ids_obj:
            farmer_cluster = FarmerClusterSeasonMap.objects.get(farmer_id=farmer, season_id=season_id).cluster_id
            if superviosr_cluster != farmer_cluster:
                return False
    return True
    
# 8. To check both agent and superviosr same cluster
def check_both_agent_and_superviosr_same_cluster():
    print('To check both agent and superviosr same cluster')
    superviosr_ids_obj = list(UserClusterMap.objects.filter(season_id=season_id, user__userprofile__user_type_id=5).values_list('user_id', flat=True))
    temp_dict={}

    for superviosr in superviosr_ids_obj:
        superviosr_cluster = UserClusterMap.objects.get(season_id=season_id, user_id=superviosr).cluster_id
        agent_ids = list(AgentSupervisorSeasonMap.objects.filter(season_id=season_id, supervisor_id=superviosr).values_list('agent_id', flat=True))
        for agent_id in agent_ids:
            agent_cluster = UserClusterMap.objects.get(season_id=season_id, user_id=agent_id).cluster_id
            if superviosr_cluster != agent_cluster:
                return False
    return True
 

# 9. To check_seasonal_farmer_code
def check_seasonal_farmer_code():
    print('to check_seasonal_farmer_code')
    farmer_obj = FarmerClusterSeasonMap.objects.filter(season_id=season_id)
    codes=[]
    master_dict={}
    for code in FarmerClusterSeasonMap.objects.filter(season_id=season_id):
        farmer_code = code.seasonal_farmer_code

        farmer_cluster_letter = FarmerClusterSeasonMap.objects.get(season_id=season_id, seasonal_farmer_code=farmer_code).cluster.name[:1]

        cluster_id = FarmerClusterSeasonMap.objects.get(season_id=season_id, seasonal_farmer_code=farmer_code).cluster.id
        farmer_supervisor_letter = UserClusterMap.objects.get(season=3, cluster_id=cluster_id, user__userprofile__user_type_id=5).user.username[:1]
        agent_id = AgentFarmerMap.objects.get(farmer__seasonal_farmer_code=farmer_code).agent_id

        farmer_unicode_letter = UserClusterMap.objects.get(season_id=season_id, user_id=agent_id).unique_code
        gen_farmer_code_letters = farmer_cluster_letter.upper() + farmer_supervisor_letter.upper() + farmer_unicode_letter.upper()

        split_from_farme_code = farmer_code[2:5]

        if not gen_farmer_code_letters==split_from_farme_code:
            return False
    return True

# 11. check aadhaar id profile matching
def aadhaar_id_userprofile_count():
    aadhaar_ids = list(UserProfile.objects.exclude(aadhaar_number__isnull=True).exclude(aadhaar_number='11111111111').exclude(aadhaar_number='111111111111').values_list('aadhaar_number', flat=True))
    aadhaar_dict={}
    for item in aadhaar_ids:
        aadhaar_dict[item] = []
        user_profile_obj = UserProfile.objects.filter(aadhaar_number=item)
        for id in user_profile_obj:
            aadhaar_dict[item].append(id.user.first_name)
    for agent_id in aadhaar_dict.keys():
        if len(aadhaar_dict[agent_id]) >= 2:
            return False
    return True

            
# 12. To check one agent in multiple cluster
def check_one_agent_in_multiple_cluster():
    print('To check one agent in multiple cluster')
    agent_ids = list(set(UserClusterMap.objects.filter(season_id=season_id, user__userprofile__user_type_id=6).values_list('user_id', flat=True)))
    temp={}
    cluster_ids = list(ClusterSeasonMap.objects.filter(season_id=season_id).values_list('cluster_id', flat=True))
    for agent_id in agent_ids:
        agent_count = UserClusterMap.objects.filter(season_id=season_id, user_id=agent_id, cluster_id__in=cluster_ids).count()
        if agent_count >= 2:
            return False
    return True


# 13. To check farmer id in UserFarmerMap
def check_farmer_id_in_user_farmer_map():
    print('To check farmer id in UserFarmerMap')
    cluster_ids = list(ClusterSeasonMap.objects.filter(season_id=season_id).values_list('cluster_id', flat=True))
    farmer_ids = list(FarmerClusterSeasonMap.objects.filter(season_id=season_id, cluster_id__in=cluster_ids).values_list('farmer_id', flat=True))
    for farmer_id in farmer_ids:
        if not UserFarmerMap.objects.filter(farmer__season_id=season_id, farmer__farmer_id=farmer_id).exists():
            return False
    return True            
            
# 14. To check farmer id in AgentFarmerMap
def check_farmer_id_in_agent_farmer_map():
    cluster_ids = list(ClusterSeasonMap.objects.filter(season_id=season_id).values_list('cluster_id', flat=True))
    farmer_ids = list(FarmerClusterSeasonMap.objects.filter(season_id=season_id, cluster_id__in=cluster_ids).values_list('farmer_id', flat=True))
    farmer_ids
    for farmer_id in farmer_ids:
        if not AgentFarmerMap.objects.filter(farmer__season_id=season_id, farmer__farmer_id=farmer_id).exists():
            return False
    return True
check_test_cases()

To check cluster and supervisor count
To check agent_count_in_cluster_and_supervisor
success
